<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC2_FD002_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
%pip install scikeras;
%pip install -U tensorflow-addons;
%pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 28.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 9.7 MB/s 


In [ ]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import datetime

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.model_selection import GroupKFold

from sklearn.base import BaseEstimator,RegressorMixin

In [ ]:
from skopt import BayesSearchCV

from skopt.space.space import Categorical, Integer, Real

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Masking

from tensorflow.keras.metrics import RootMeanSquaredError as RMSE

from keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.optimizers.schedules import ExponentialDecay

import tensorflow_addons as tfa
from tensorflow_addons.metrics import RSquare as R2

from scikeras.wrappers import KerasRegressor

In [ ]:
sns.set_palette('colorblind')

In [ ]:
# Reproducibility
seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
tf.config.experimental.enable_op_determinism()

In [ ]:
# Remove some tf warnings
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

# Data Preparation

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    IN_COLAB = True
    folder="/content/"

    # Dataset Download 
    os.system('git clone https://github.com/arthursl12/dataset_2')
    os.system('mv /content/dataset_2/CMaps /content/CMaps')
    os.system('mv /content/dataset_2/data_processing /content/data_processing')
    os.system('rm -rf dataset_2')
else:
    print('Not running on CoLab')
    IN_COLAB = False
    folder="CMaps/"
    %cd dataset_2/

Running on CoLab


In [ ]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [ ]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [ ]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(2, folder='CMaps/')
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,...,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,...,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
2,1,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,...,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,...,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
4,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,...,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53754,260,312,20.0037,0.7000,100.0,491.19,608.79,1495.60,1269.51,9.35,...,314.05,2389.02,8169.64,9.3035,0.03,369,2324,100.00,24.36,14.5189
53755,260,313,10.0022,0.2510,100.0,489.05,605.81,1514.32,1324.12,10.52,...,371.22,2388.42,8245.36,8.7586,0.03,374,2319,100.00,28.10,16.9454
53756,260,314,25.0041,0.6200,60.0,462.54,537.48,1276.24,1057.92,7.05,...,163.74,2030.33,7971.25,11.0657,0.02,310,1915,84.93,14.19,8.5503
53757,260,315,25.0033,0.6220,60.0,462.54,537.84,1272.95,1066.30,7.05,...,164.37,2030.35,7972.47,11.0537,0.02,311,1915,84.93,14.05,8.3729


## Preprocessing

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [ ]:
test.shape, y_test.shape

((33991, 26), (259, 1))

In [ ]:
test_last = proc.transform_test_keep_setting(test)
test_last.head()

,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,s_5,s_6,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,10.0076,0.2501,100.0,489.05,605.42,1515.00,1325.07,10.52,15.50,393.58,...,370.87,2388.32,8167.06,8.7456,0.03,371,2319,100.0,28.30,17.0934
1,0.0018,0.0000,100.0,518.67,642.67,1591.67,1418.17,14.62,21.61,553.36,...,521.10,2388.12,8138.12,8.4248,0.03,393,2388,100.0,38.82,23.3463
2,35.0015,0.8412,100.0,449.44,555.86,1370.62,1135.59,5.48,8.00,194.58,...,183.11,2388.07,8071.23,9.3094,0.02,332,2223,100.0,14.75,8.9589
3,20.0032,0.7000,100.0,491.19,607.99,1487.94,1257.49,9.35,13.66,334.39,...,314.88,2388.12,8062.39,9.2349,0.02,365,2324,100.0,24.22,14.6814
4,42.0055,0.8400,100.0,445.00,550.81,1358.95,1140.34,3.91,5.72,138.42,...,130.82,2389.06,8140.94,9.3964,0.02,333,2212,100.0,10.34,6.3601


In [ ]:
X_test = test_last

### Remaining Useful Life (RUL)

In [ ]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,148
1,1,2,147
2,1,3,146
3,1,4,145
4,1,5,144


## Attributes and target separation

In [ ]:
X_train, y_train = proc.X_y_train_divide_with_settings(train)

In [ ]:
y_train.head()

,RUL
0,148
1,147
2,146
3,145
4,144


In [ ]:
X_train.head()

,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,s_5,s_6,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,8.00,194.64,...,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
1,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,5.71,138.51,...,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
2,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,9.02,175.71,...,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
3,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,5.71,138.46,...,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
4,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,9.03,175.05,...,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286


## Training and Evaluation functions

In [ ]:
eval = Evaluation()

In [ ]:
search = HyperparameterSearch()

# LSTM Construction

## Callbacks

In [ ]:
# Early Stopping Callback
es = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                      patience=5, restore_best_weights=True)

In [ ]:
# Printing Callback
def printLog(epoch, logs):
    print(
        f"E {epoch+1}\t: loss={logs['loss']:.3f}, "+
        f"rmse={logs['root_mean_squared_error']:.3f}, "+
        f"r2={logs['r_square']:.3f}; "+
        f"v_loss={logs['val_loss']:.3f}, "+
        f"v_rmse={logs['val_root_mean_squared_error']:.3f}, "+
        f"v_r2={logs['val_r_square']:.3f}; "
    )

printerCallback = LambdaCallback(on_epoch_end=printLog)

## Constants

In [ ]:
# X_train must include indices
train3 = train.copy()
X_train_ = train3.drop(columns=["RUL"])

In [ ]:
INPUT_SHAPE = 0

## Wrapper

In [ ]:
from sklearn.metrics import r2_score

class LSTMWrapperRegressor(BaseEstimator,RegressorMixin):
    def __init__(self, basemodel=None, clip_y=-1, seq_length=40,
                 include_settings=False, poly_degree=1,
                 scaler=StandardScaler()):
        # Base parameters
        self.basemodel = basemodel
        self.clip_y = clip_y
        self.seq_length = seq_length
        self.poly_degree = poly_degree
        self.include_settings = include_settings

        # Column indexers
        self.feature_cols = sensors_cols
        if(include_settings):
            # self.seq_cols = settings_cols + self.cols
            self.feature_cols = settings_cols + self.feature_cols
        self.seq_cols = ["time"] + self.feature_cols
        self.base_feature_cols = self.feature_cols

        # Scaler and PolyFeatures transformers
        self.scaler = scaler
        self.polyft = PolynomialFeatures(degree=self.poly_degree, 
                                         include_bias=False)



    def fit(self, X=None, y=None):
        # Merge features and target again
        data = X.copy()
        data["RUL"] = y

        # Apply polynomial features and add them to the dataframe
        transf = self.polyft.fit_transform(data[self.base_feature_cols])
        transf = pd.DataFrame(transf, columns=
                              self.polyft.get_feature_names_out(),
                              index=data.index)
        self.feature_cols = list(self.polyft.get_feature_names_out())
        self.seq_cols = ["time"] + self.feature_cols
        data[self.feature_cols] = transf

        # Scale the data
        data[self.feature_cols] = \
                            self.scaler.fit_transform(data[self.feature_cols])

        # Transform into time series
        X_train = self.gen_X_wrapper(data,self.seq_length,self.seq_cols)
        # print(data.shape,X_train.shape, X.shape)

        # Clip and transform labels
        data2 = data.copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_train = self.gen_y_wrapper(data2,self.seq_length,["RUL"])

        # Update input shape for future use
        global INPUT_SHAPE
        # print(INPUT_SHAPE, X_train.shape)
        INPUT_SHAPE = (X_train.shape[1],X_train.shape[2])

        # Fit model
        # print(X_train.shape, y_train.shape)
        self.basemodel.fit(X_train,y_train)
        return self
        
    def predict(self, X=None):
        # Perform transformation, if not done
        if (len(X.shape) < 3):
            data = X.copy()

            # Apply polynomial features
            transf = self.polyft.transform(data[self.base_feature_cols])
            transf = pd.DataFrame(transf, 
                                  columns=self.polyft.get_feature_names_out(),
                                  index=data.index)
            # data = pd.concat([data,transf], axis=1)
            data[self.feature_cols] = transf

            # Scale the data
            data[self.feature_cols] = \
                            self.scaler.transform(data[self.feature_cols])
            
            # Transform into time series
            X_train = self.gen_X_wrapper(data,self.seq_length,self.seq_cols)
        else:
            X_train = X
        return self.basemodel.predict(X_train)

    def score(self, X, y, sample_weight=None):
        # Merge features and target again
        data = X.copy()
        data["RUL"] = y

        # Apply polynomial features
        transf = self.polyft.transform(data[self.base_feature_cols])
        transf = pd.DataFrame(transf, 
                              columns=self.polyft.get_feature_names_out(),
                              index=data.index)
        # data = pd.concat([data,transf], axis=1)
        data[self.feature_cols] = transf

        # Scale the data (with train data parameters)
        data[self.feature_cols] = \
                        self.scaler.transform(data[self.feature_cols])
        
        # Transform into time series
        X_test = self.gen_X_wrapper(data,self.seq_length,self.seq_cols)
        # print(data.shape,X_test.shape, X.shape)

        # Clip and transform labels
        data2 = data.copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_test = self.gen_y_wrapper(data2,self.seq_length,["RUL"])

        # Predict on test data
        y_pred = self.predict(X_test)
        return r2_score(y_test, y_pred, sample_weight=sample_weight)

    def gen_X_data(self, df, sequence_length, columns, mask_value=-99.):
        if df.shape[0] < sequence_length:
            # print("\t Not enough sequence:",df.shape[0]," < ",sequence_length)
            data = np.full(shape=(sequence_length, len(columns)), fill_value=mask_value) # pad
            idx = data.shape[0] - df.shape[0]
            data[idx:,:] = df[columns].values  # fill with available data
        else:
            data = df[columns].values
            
        # # specifically yield the last possible sequence
        # stop = num_elements = data_matrix.shape[0]
        # start = stop - sequence_length
        # for i in list(range(1)):
        #     yield data_matrix[start:stop, :]



        # data = df[columns].values
        num_elements = data.shape[0]

        # -1 and +1 because of Python indexing
        for start, stop in zip(range(0, num_elements-(sequence_length-1)), 
                               range(sequence_length, num_elements+1)):
            yield data[start:stop, :]

    def gen_X_wrapper(self, df, sequence_length, columns, 
                      unit_nrs=np.array([]), idx_col="unit_number"):
        # print(">> Wrapper called. df shape:",df.shape)
        if unit_nrs.size <= 0:
            unit_nrs = df[idx_col].unique()
            
        data_gen = (list(self.gen_X_data(df[df[idx_col]==unit_nr], 
                                         sequence_length, columns))
                for unit_nr in unit_nrs)
        # print("\tdatagen len:",len(data_gen))
        data_array = np.concatenate(list(data_gen)).astype(np.float32)
        # print("\tdata_array.shape:",data_array.shape)
        return data_array

    def gen_y(self, df, sequence_length, label):
        data_matrix = df[label].values
        num_elements = data_matrix.shape[0]

        # -1 because I want to predict the rul of that last row in the sequence, 
        # not the next row
        return data_matrix[sequence_length-1:num_elements, :]  

    def gen_y_wrapper(self, df, sequence_length, label, 
                      unit_nrs=np.array([]), idx_col="unit_number"):
        # print(">> Y Wrapper called. df shape:",df.shape)
        if unit_nrs.size <= 0:
            unit_nrs = df[idx_col].unique()
            
        label_gen = [self.gen_y(df[df[idx_col]==unit_nr], 
                                sequence_length, label) 
                    for unit_nr in unit_nrs]
        # print("\tlabelgen len:",len(label_gen))
        label_array = np.concatenate(label_gen).astype(np.float32)
        # print("\tlabel_array.shape:",label_array.shape)
        return label_array

## Test Data

In [ ]:
def gen_test_data(df, sequence_length, columns, mask_value):
    if df.shape[0] < sequence_length:
        data_matrix = np.full(shape=(sequence_length, len(columns)), fill_value=mask_value) # pad
        idx = data_matrix.shape[0] - df.shape[0]
        data_matrix[idx:,:] = df[columns].values  # fill with available data
    else:
        data_matrix = df[columns].values
        
    # specifically yield the last possible sequence
    stop = num_elements = data_matrix.shape[0]
    start = stop - sequence_length
    for i in list(range(1)):
        yield data_matrix[start:stop, :]

In [ ]:
def gen_test_wrapper(X_test_scaled, sequence_length, cols, idx_col="unit_number"): 
    data_gen = (
        list(gen_test_data(X_test_scaled[X_test_scaled[idx_col]==unit_nr], 
                           sequence_length, cols, -99.))
            for unit_nr in X_test_scaled[idx_col].unique())
    data_array = np.concatenate(list(data_gen)).astype(np.float32)
    return data_array

In [ ]:
def scale_test(test,model):
    test2 = test.copy()

    # Apply polynomial features
    transf = model.polyft.transform(test2[model.base_feature_cols])
    transf = pd.DataFrame(transf, 
                          columns=model.polyft.get_feature_names_out(),
                          index=test2.index)
    newcols = model.polyft.get_feature_names_out()
    test2[newcols] = transf 

    # Scale the data (with train data parameters)
    test2[model.feature_cols] = \
                    model.scaler.transform(test2[model.feature_cols])
    return test2

## Constructor

In [ ]:
def create_model(optim=Adam, learning_rate=1e-3, 
                 layer1=32  , activation1="tanh"    , dropout1=0.1,
                 layer2=None, activation2="tanh"    , dropout2=0.1,
                 layer3=None, activation3="tanh"    , dropout3=0.1,
                 second_dense=True,
                 print_summary=False, loss='mean_squared_error',
                 metrics=[tf.keras.metrics.MeanSquaredError()]):
    model = Sequential()

    # Input-masked layer
    model.add(Masking(mask_value=-99., input_shape=INPUT_SHAPE))
    
    if (layer2 is None and layer3 is None):
        # Single LSTM layer
        model.add(LSTM(layer1, activation=activation1))
        model.add(Dropout(dropout1))
    elif (layer2 is not None and layer3 is None):
        # LSTM-Dense
        model.add(LSTM(layer1, activation=activation1))
        model.add(Dropout(dropout1))
        model.add(Dense(layer2, activation=activation2))
        model.add(Dropout(dropout2))
    elif (layer2 is not None and layer3 is not None and second_dense==False):
        # LSTM-LSTM-Dense
        model.add(LSTM(layer1, activation=activation1, return_sequences=True))
        model.add(Dropout(dropout1))
        model.add(LSTM(layer2, activation=activation2))
        model.add(Dropout(dropout2))
        model.add(Dense(layer3, activation=activation3))
        model.add(Dropout(dropout3))
    elif (layer2 is not None and layer3 is not None and second_dense==True):
        # LSTM-Dense-Dense
        model.add(LSTM(layer1, activation=activation1))
        model.add(Dropout(dropout1))
        model.add(Dense(layer2, activation=activation2))
        model.add(Dropout(dropout2))
        model.add(Dense(layer3, activation=activation3))
        model.add(Dropout(dropout3))

    # Output Layer
    model.add(Dense(1))

    model.compile(loss=loss, optimizer=optim(learning_rate=learning_rate), 
                  metrics=metrics)
    
    if(print_summary): model.summary()
    return model

# LSTM 1-layer 

## Linear RUL

Score: 0.5816177553633815  
Test: 0.625
```
('basemodel__batch_size', 32),
('basemodel__epochs', 35),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__dropout1', 0.9),
('basemodel__model__layer1', 512),
('basemodel__model__learning_rate', 0.0001),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('scaler', StandardScaler()),
('seq_length', 100)
```



In [ ]:
SEQ_LENGTH=100
CLIP=-1

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=32,
                           epochs=35,
                           model__activation1='tanh',
                           model__dropout1=0.9, 
                           model__layer1=512, 
                           model__learning_rate=0.0001,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [ ]:
model.fit(X_train_, y_train)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 100, 25)           0         
                                                                 
 lstm (LSTM)                 (None, 512)               1101824   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 1,102,337
Trainable params: 1,102,337
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=3586.621, rmse=59.888, r2=-0.647; v_loss=4061.905, v_rmse=63.733, v_r2=-0.391; 
E 2	: loss=2453.961, rmse=49.537, r2=-0.127; v_loss=3340.924, v_rmse=57.801, v_r2=-0.14

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fd7e8fdd450>, <keras.callbacks.LambdaCallback object at 0x7fd7e8fdda90>], epochs=35, model=<function create_model at 0x7fd7e8f44560>, model__activation1='tanh', model__dropout1=0.9, model__layer1=512, model__learning_rate=0.0001, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7fd7e9021810>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fd842bc3850>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                     include_settings=True, seq_length=100)

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.625,RMSE=-32.923
Finished: 2022-10-25 19:55:52.973003


## Non-Linear RUL (More Epochs)

Score: 0.6487511063058244  
Test: 0.835
```
('basemodel__batch_size', 262),
('basemodel__epochs', 50),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__dropout1', 0.9),
('basemodel__model__layer1', 512),
('basemodel__model__learning_rate', 0.0001),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.13191782981852057),
('clip_y', 140),
('include_settings', True),
('scaler', StandardScaler()),
('seq_length', 75)
```


In [ ]:
SEQ_LENGTH=75
CLIP=140

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=262,
                           epochs=70,
                           model__activation1='tanh',
                           model__dropout1=0.9, 
                           model__layer1=512, 
                           model__learning_rate=0.0001,
                           model__optim=Adam,
                           validation_split=0.13191782981852057, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [ ]:
model.fit(X_train_, y_train)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_2 (Masking)         (None, 75, 25)            0         
                                                                 
 lstm_2 (LSTM)               (None, 512)               1101824   
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 513       
                                                                 
Total params: 1,102,337
Trainable params: 1,102,337
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=5544.692, rmse=74.463, r2=-1.991; v_loss=5043.817, v_rmse=71.020, v_r2=-1.467; 
E 2	: loss=3876.520, rmse=62.262, r2=-1.091; v_loss=3990.112, v_rmse=63.167, v_r2=-0.

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=262, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fd7e8fdd450>, <keras.callbacks.LambdaCallback object at 0x7fd7e8fdda90>], epochs=70, model=<function create_model at 0x7fd7e8f44560>, model__activation1='tanh', model__dropout1=0.9, model__layer1=512, model__learning_rate=0.0001, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7fd77cac6110>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fd77ccf5050>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.13191782981852057, verbose=0),
                     clip_y=140, include_settings=True, seq_length=75)

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.835,RMSE=-19.010
Finished: 2022-10-25 20:10:53.361820


# LSTM-Dense-1 

## Linear RUL




Score: 0.5709418653283571  
Test: 0.730
```
('basemodel__batch_size', 32),
('basemodel__epochs', 43),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'elu'),
('basemodel__model__dropout1', 0.4523423204279986),
('basemodel__model__dropout2', 0.6519444578987227),
('basemodel__model__layer1', 492),
('basemodel__model__layer2', 53),
('basemodel__model__learning_rate', 0.0001),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__model__second_dense', True),
('basemodel__validation_split', 0.22182563254238585),
('include_settings', True),
('scaler', StandardScaler()),
('seq_length', 53)
```




In [ ]:
SEQ_LENGTH=53
CLIP=-1

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=32,
                           epochs=43,
                           model__activation1='tanh',
                           model__activation2='elu',
                           model__dropout1=0.4523423204279986, 
                           model__dropout2=0.6519444578987227, 
                           model__layer1=492, 
                           model__layer2=53, 
                           model__learning_rate=0.0001,
                           model__optim=Adam,
                           model__second_dense=True,
                           validation_split=0.22182563254238585, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [ ]:
model.fit(X_train_, y_train)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_3 (Masking)         (None, 53, 25)            0         
                                                                 
 lstm_3 (LSTM)               (None, 492)               1019424   
                                                                 
 dropout_3 (Dropout)         (None, 492)               0         
                                                                 
 dense_3 (Dense)             (None, 53)                26129     
                                                                 
 dropout_4 (Dropout)         (None, 53)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 54        
                                                                 
Total params: 1,045,607
Trainable params: 1,045,607
No

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fd7e8fdd450>, <keras.callbacks.LambdaCallback object at 0x7fd7e8fdda90>], epochs=43, model=<function create_model at 0x7fd7e8f44560>, model__activation1='tanh', model__activation2='elu', model__dropout1=0.4523423204279986, model__dropout2=0.6519444578...ss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7fd7583cc0d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fd7583d36d0>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, model__second_dense=True, print_summary=True, validation_split=0.22182563254238585, verbose=0),
                     include_settings=True, seq_length=53)

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.730,RMSE=-27.967
Finished: 2022-10-25 20:21:55.227530


## Non-Linear RUL (More Epochs)


Score: 0.7530758636321139  
Test: 0.895
```
('basemodel__batch_size', 287),
('basemodel__epochs', 28),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'elu'),
('basemodel__model__dropout1', 0.13678820565973349),
('basemodel__model__dropout2', 0.9),
('basemodel__model__layer1', 336),
('basemodel__model__layer2', 77),
('basemodel__model__learning_rate', 0.0040050838967013785),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__model__second_dense', True),
('basemodel__validation_split', 0.1494847733663106),
('clip_y', 124),
('include_settings', True),
('scaler', StandardScaler()),
('seq_length', 73)
```


In [ ]:
SEQ_LENGTH=73
CLIP=124

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=287,
                           epochs=50,
                           model__activation1='tanh',
                           model__activation2='elu',
                           model__dropout1=0.13678820565973349, 
                           model__dropout2=0.9, 
                           model__layer1=336, 
                           model__layer2=77, 
                           model__learning_rate=0.0040050838967013785,
                           model__optim=Adam,
                           model__second_dense=True,
                           validation_split=0.1494847733663106, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [ ]:
model.fit(X_train_, y_train)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_5 (Masking)         (None, 73, 25)            0         
                                                                 
 lstm_5 (LSTM)               (None, 336)               486528    
                                                                 
 dropout_7 (Dropout)         (None, 336)               0         
                                                                 
 dense_7 (Dense)             (None, 77)                25949     
                                                                 
 dropout_8 (Dropout)         (None, 77)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 78        
                                                                 
Total params: 512,555
Trainable params: 512,555
Non-tr

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=287, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fd7e8fdd450>, <keras.callbacks.LambdaCallback object at 0x7fd7e8fdda90>], epochs=50, model=<function create_model at 0x7fd7e8f44560>, model__activation1='tanh', model__activation2='elu', model__dropout1=0.13678820565973349, model__dropout2=0.9, model_...odel__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7fd7583c7450>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fd6a1101050>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, model__second_dense=True, print_summary=True, validation_split=0.1494847733663106, verbose=0),
                     clip_y=124, include_settings=True, seq_length=73)

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.895,RMSE=-13.821
Finished: 2022-10-25 20:30:14.001723


# LSTM-Dense-2

## Linear RUL (More Epochs, ES)


Score: 0.5860971746607238  
Test: 0.754
```
('basemodel__batch_size', 264),
('basemodel__epochs', 23),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'sigmoid'),
('basemodel__model__activation3', 'relu'),
('basemodel__model__dropout1', 0.21534955706280656),
('basemodel__model__dropout2', 0.7741992560602),
('basemodel__model__dropout3', 0.4418655267370424),
('basemodel__model__layer1', 395),
('basemodel__model__layer2', 330),
('basemodel__model__layer3', 145),
('basemodel__model__learning_rate', 0.009202704024775169),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__model__second_dense', True),
('basemodel__validation_split', 0.20435450812720612),
('include_settings', True),
('scaler', StandardScaler()),
('seq_length', 70)
```





In [ ]:
SEQ_LENGTH=70
CLIP=-1

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=264,
                           epochs=50,
                           model__activation1='tanh',
                           model__activation2='sigmoid',
                           model__activation3='relu',
                           model__dropout1=0.21534955706280656, 
                           model__dropout2=0.7741992560602,
                           model__dropout3=0.4418655267370424, 
                           model__layer1=395, 
                           model__layer2=330, 
                           model__layer3=145, 
                           model__learning_rate=0.009202704024775169,
                           model__optim=Adam,
                           model__second_dense=True,
                           validation_split=0.20435450812720612, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [ ]:
model.fit(X_train_, y_train)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_7 (Masking)         (None, 70, 25)            0         
                                                                 
 lstm_7 (LSTM)               (None, 395)               665180    
                                                                 
 dropout_12 (Dropout)        (None, 395)               0         
                                                                 
 dense_12 (Dense)            (None, 330)               130680    
                                                                 
 dropout_13 (Dropout)        (None, 330)               0         
                                                                 
 dense_13 (Dense)            (None, 145)               47995     
                                                                 
 dropout_14 (Dropout)        (None, 145)              

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=264, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fd7e8fdd450>, <keras.callbacks.LambdaCallback object at 0x7fd7e8fdda90>], epochs=50, model=<function create_model at 0x7fd7e8f44560>, model__activation1='tanh', model__activation2='sigmoid', model__activation3='relu', model__dropout1=0.21534955706280...ss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7fd69be60f10>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fd69be61110>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, model__second_dense=True, print_summary=True, validation_split=0.20435450812720612, verbose=0),
                     include_settings=True, seq_length=70)

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.754,RMSE=-26.654
Finished: 2022-10-25 20:43:38.876189


## Non-Linear RUL 


Score: 0.8114717682301014  
Test: 0.890
```
('basemodel__batch_size', 32),
('basemodel__epochs', 47),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'tanh'),
('basemodel__model__activation3', 'sigmoid'),
('basemodel__model__dropout1', 0.1),
('basemodel__model__dropout2', 0.2141175093000496),
('basemodel__model__dropout3', 0.7714899358963251),
('basemodel__model__layer1', 16),
('basemodel__model__layer2', 16),
('basemodel__model__layer3', 100),
('basemodel__model__learning_rate', 0.000567080109365328),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__model__second_dense', True),
('basemodel__validation_split', 0.11371723233432361),
('clip_y', 106),
('include_settings', True),
('scaler', StandardScaler()),
('seq_length', 44)
```




In [ ]:
SEQ_LENGTH=44
CLIP=106

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=32,
                           epochs=47,
                           model__activation1='tanh',
                           model__activation2='tanh',
                           model__activation3='sigmoid',
                           model__dropout1=0.1, 
                           model__dropout2=0.2141175093000496,
                           model__dropout3=0.7714899358963251, 
                           model__layer1=16, 
                           model__layer2=16, 
                           model__layer3=100, 
                           model__learning_rate=0.000567080109365328,
                           model__optim=Adam,
                           model__second_dense=True,
                           validation_split=0.11371723233432361, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [ ]:
model.fit(X_train_, y_train)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_11 (Masking)        (None, 44, 25)            0         
                                                                 
 lstm_11 (LSTM)              (None, 16)                2688      
                                                                 
 dropout_24 (Dropout)        (None, 16)                0         
                                                                 
 dense_24 (Dense)            (None, 16)                272       
                                                                 
 dropout_25 (Dropout)        (None, 16)                0         
                                                                 
 dense_25 (Dense)            (None, 100)               1700      
                                                                 
 dropout_26 (Dropout)        (None, 100)             

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fd7e8fdd450>, <keras.callbacks.LambdaCallback object at 0x7fd7e8fdda90>], epochs=47, model=<function create_model at 0x7fd7e8f44560>, model__activation1='tanh', model__activation2='tanh', model__activation3='sigmoid', model__dropout1=0.1, model__dropou...del__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7fd6937c4810>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fd6937c4a10>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, model__second_dense=True, print_summary=True, validation_split=0.11371723233432361, verbose=0),
                     clip_y=106, include_settings=True, seq_length=44)

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.901,RMSE=-11.743
Finished: 2022-10-25 21:24:24.591407


# LSTM-LSTM-Dense 

## Linear RUL



Score: 0.4572066088840683  
Test: 0.
```
('basemodel__batch_size', 413),
('basemodel__epochs', 39),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'tanh'),
('basemodel__model__activation3', 'elu'),
('basemodel__model__dropout1', 0.8027344701984965),
('basemodel__model__dropout2', 0.7569408037894426),
('basemodel__model__dropout3', 0.12896783869092998),
('basemodel__model__layer1', 367),
('basemodel__model__layer2', 478),
('basemodel__model__layer3', 22),
('basemodel__model__learning_rate', 0.0068513804632596),
('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
('basemodel__model__second_dense', False),
('basemodel__validation_split', 0.6070472478342536),
('include_settings', True),
('scaler', MinMaxScaler()),
('seq_length', 32)
```





In [ ]:
SEQ_LENGTH=32
CLIP=-1

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=MinMaxScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=413,
                           epochs=39,
                           model__activation1='tanh',
                           model__activation2='tanh',
                           model__activation3='elu',
                           model__dropout1=0.8027344701984965, 
                           model__dropout2=0.7569408037894426,
                           model__dropout3=0.12896783869092998, 
                           model__layer1=367, 
                           model__layer2=478, 
                           model__layer3=22, 
                           model__learning_rate=0.0068513804632596,
                           model__optim=RMSprop,
                           model__second_dense=False,
                           validation_split=0.6070472478342536, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [ ]:
model.fit(X_train_, y_train)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 32, 25)            0         
                                                                 
 lstm (LSTM)                 (None, 32, 367)           576924    
                                                                 
 dropout (Dropout)           (None, 32, 367)           0         
                                                                 
 lstm_1 (LSTM)               (None, 478)               1617552   
                                                                 
 dropout_1 (Dropout)         (None, 478)               0         
                                                                 
 dense (Dense)               (None, 22)                10538     
                                                                 
 dropout_2 (Dropout)         (None, 22)                0

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=413, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f8d0c16a590>, <keras.callbacks.LambdaCallback object at 0x7f8d0c16e690>], epochs=39, model=<function create_model at 0x7f8d0c17cf80>, model__activation1='tanh', model__activation2='tanh', model__activation3='elu', model__dropout1=0.8027344701984965, m...ras.metrics.metrics.RootMeanSquaredError object at 0x7f8d0c16d210>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f8d66adb410>], model__optim=<class 'keras.optimizers.optimizer_v2.rmsprop.RMSprop'>, model__second_dense=False, print_summary=True, validation_split=0.6070472478342536, verbose=0),
                     include_settings=True, scaler=MinMaxScaler(),
                     seq_length=32)

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.488,RMSE=-38.494
Finished: 2022-10-30 10:58:37.479329


## Non-Linear RUL

Score: 0.5770400313627971  
Test: 0.
```
('basemodel__batch_size', 475),
('basemodel__epochs', 38),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'tanh'),
('basemodel__model__activation3', 'selu'),
('basemodel__model__dropout1', 0.41871805965088293),
('basemodel__model__dropout2', 0.3421999536441197),
('basemodel__model__dropout3', 0.6177826332606998),
('basemodel__model__layer1', 440),
('basemodel__model__layer2', 184),
('basemodel__model__layer3', 376),
('basemodel__model__learning_rate', 0.0038903406569891345),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__model__second_dense', False),
('basemodel__validation_split', 0.7313863245835751),
('clip_y', 100),
('include_settings', True),
('scaler', StandardScaler()),
('seq_length', 44)
```


In [ ]:
SEQ_LENGTH=44
CLIP=100

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=475,
                           epochs=38,     
                           model__activation1='tanh',
                           model__activation2='tanh',
                           model__activation3='selu',
                           model__dropout1=0.41871805965088293, 
                           model__dropout2=0.3421999536441197,
                           model__dropout3=0.6177826332606998, 
                           model__layer1=440, 
                           model__layer2=184, 
                           model__layer3=376, 
                           model__learning_rate=0.0038903406569891345,
                           model__optim=Adam,
                           model__second_dense=False,
                           validation_split=0.7313863245835751, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [ ]:
model.fit(X_train_, y_train)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_1 (Masking)         (None, 44, 25)            0         
                                                                 
 lstm_2 (LSTM)               (None, 44, 440)           820160    
                                                                 
 dropout_3 (Dropout)         (None, 44, 440)           0         
                                                                 
 lstm_3 (LSTM)               (None, 184)               460000    
                                                                 
 dropout_4 (Dropout)         (None, 184)               0         
                                                                 
 dense_2 (Dense)             (None, 376)               69560     
                                                                 
 dropout_5 (Dropout)         (None, 376)              

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=475, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f8d0c16a590>, <keras.callbacks.LambdaCallback object at 0x7f8d0c16e690>], epochs=38, model=<function create_model at 0x7f8d0c17cf80>, model__activation1='tanh', model__activation2='tanh', model__activation3='selu', model__dropout1=0.41871805965088293...del__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f8c9f9c2410>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f8c9f9c2690>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, model__second_dense=False, print_summary=True, validation_split=0.7313863245835751, verbose=0),
                     clip_y=100, include_settings=True, seq_length=44)

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.582,RMSE=-22.862
Finished: 2022-10-30 11:02:43.976159
